In [ ]:
import smtplib
from email.mime.text import MIMEText

def kirim_email_laporan(penerima, subjek, isi_pesan):
    # Konfigurasi Akun Pengirim (Ganti dengan data Anda)
    smtp_server = "smtp.gmail.com" # Contoh untuk Gmail
    smtp_port = 587                  # Port TLS/STARTTLS
    pengirim_email = "fikri.zainul.sulaeman@gmail.com"
    password = "eepsugfylpvanvrn" # Gunakan Kata Sandi Aplikasi, bukan kata sandi akun

    try:
        # Membuat objek MIMEText untuk isi email
        msg = MIMEText(isi_pesan)
        msg['Subject'] = subjek
        msg['From'] = pengirim_email
        msg['To'] = penerima

        # Menghubungkan ke server SMTP
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Mengaktifkan enkripsi
            server.login(pengirim_email, password)
            server.sendmail(pengirim_email, penerima, msg.as_string())

        print(f"Email laporan berhasil dikirim ke {penerima}.")

    except Exception as e:
        print(f"Gagal mengirim email: {e}")

# Contoh penggunaan (jangan jalankan tanpa konfigurasi yang benar)
kirim_email_laporan(
    penerima="fikrizainul@yahoo.com",
    subjek="Laporan Penjualan Harian",
    isi_pesan="Penjualan hari ini mencapai Rp 5.000.000. Cek data lengkap di lampiran."
)

Email laporan berhasil dikirim ke fikrizainul@yahoo.com.


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

def kirim_email_dengan_lampiran(penerima_email, subjek, isi_pesan, path_lampiran):
    # --- KONFIGURASI PENGIRIM (GANTI DENGAN DATA ANDA) ---
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    pengirim_email = "email_anda@gmail.com"
    password = "KATA_SANDI_APLIKASI_ANDA"
    # ----------------------------------------------------

    # 1. Membuat Pesan Multi-Bagian (MIMEMultipart)
    msg = MIMEMultipart()
    msg['From'] = pengirim_email
    msg['To'] = penerima_email
    msg['Subject'] = subjek
    msg.attach(MIMEText(isi_pesan, 'plain'))

    # 2. Melampirkan File
    try:
        # Membuka file dari path yang diberikan
        with open(path_lampiran, "rb") as lampiran: # 'rb' = read binary
            bagian = MIMEBase('application', 'octet-stream')
            bagian.set_payload(lampiran.read())

        encoders.encode_base64(bagian)

        # Menggunakan os.path.basename untuk mendapatkan nama file saja (myAttachment.pdf)
        nama_file = os.path.basename(path_lampiran)
        bagian.add_header(
            'Content-Disposition',
            f"attachment; filename= {nama_file}",
        )

        msg.attach(bagian)

    except FileNotFoundError:
        print(f"❌ ERROR: File lampiran tidak ditemukan di '{path_lampiran}'. Periksa kembali jalur file.")
        return
    except Exception as e:
        print(f"❌ ERROR: Terjadi masalah saat melampirkan file: {e}")
        return

    # 3. Mengirim Email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(pengirim_email, password)
            server.sendmail(pengirim_email, penerima_email, msg.as_string())

        print(f"\n✅ SUCCESS: Email berhasil dikirim ke {penerima_email} dengan lampiran: {nama_file}.")

    except Exception as e:
        print(f"\n❌ ERROR: Gagal mengirim email (Cek Konfigurasi SMTP/Password Aplikasi): {e}")


# --- CONTOH PENGGUNAAN DENGAN JALUR LENGKAP ---

# ⚠️ PENTING: Untuk path di Windows, gunakan double backslash (\\)
# atau raw string (r"...") agar Python tidak salah mengartikannya sebagai karakter escape.
# Kami akan menggunakan raw string (r"...") untuk kemudahan.
PATH_LAMPIRAN_LENGKAP = r"C:\FileAttachment\myAttachment.pdf"

kirim_email_dengan_lampiran(
    penerima_email="penerima_laporan@example.com",
    subjek="Laporan PDF dari Drive C:",
    isi_pesan="Terlampir file dari direktori lokal.",
    path_lampiran=PATH_LAMPIRAN_LENGKAP
)